In [25]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import GPT2PreTrainedModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', lang='fr')

In [6]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","cleaned_data.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [7]:
data

,Nom,Ingredients,Ustensiles,Etapes
0,soupe veloute potimarron pomme terre,2 pomme terre 2 oignon hache 3 gousse ail ha...,louche mixeur plonger blender chauffer couteau...,etape 1 enlever ecorce pepin potimarron obliga...
1,soupe oignon,4 oignon 50 gramme beurre 1 c.a. huile 1...,louche mixeur plonger blender chauffer saladie...,etape 1 pelez emincer oignon etape 2 faire rev...
2,veloute potiron,1 oignon 1 quartier potiron 1 brique creme f...,louche mixeur plonger couteau cocotte fonte po...,etape 1 couper chair potiron gros de etape 2 c...
3,veloute butternut cannelle,1 oignon 1 c.a. creme frais 1 pincee canne...,louche mixeur plonger couteau cuillere bois po...,etape 1 couper butternut gros cube etape 2 cou...
4,tarte poireau lardon,250 gramme farine 140 gramme beurre 2 c....,moule tarte four couteau pinceau rape poele sa...,etape 1 prechauffer four 210 degre c thermosta...
...,...,...,...,...
47335,liqueur genepi,40 morceau sucre 40 brin genepi 1 l alco...,entonnoir balance cuisine,etape 1 transvaser litre alcool gros bouteille...
47336,caipirinha,1 dose jus citron vert frais 15 millilitre j...,cuillere bois shaker,etape 1 shaker melanger ingredient glace pilee...
47337,kombucha maison citron gingembre,1 morceau gingembre raper 100 gramme sucre...,casserole balance cuisine,etape 1 commencer realiser sirop chauffer eau ...
47338,gin tonic,4 cl gin 2 rondelle citron 8 cl tonic,NaN,etape 1 verser glaçon verre etape 2 ajouter gi...


In [21]:
np.array(data["Ingredients"].to_list())

list

In [28]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
X = tokenizer.batch_encode_plus(np.array(data["Ingredients"].to_list()),padding=True, truncation=True)
Y = tokenizer.batch_encode_plus(np.array(data["Etapes"].to_list()),padding=True, truncation=True)

In [31]:
type(Y)

transformers.tokenization_utils_base.BatchEncoding

In [29]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'

In [10]:
data["Ingredients"].to_list()

['2 pomme terre 2 oignon hache   3 gousse ail hache   10 cl creme frais   1 c.a.c curry   1 pincee muscade 1 feuille laurier 1 potimarron 1 cube bouillon poule',
 '4 oignon   50 gramme beurre   1 c.a. huile   1 c.a. farine   25 cl vin blanc   1 l eau   6 tranche pain mie   100 gramme comte rape',
 '1 oignon   1 quartier potiron 1 brique creme frais liquide',
 '1 oignon   1 c.a. creme frais   1 pincee cannelle 1 butternut 1 cube bouillon',
 '  250 gramme farine   140 gramme beurre   2 c.a.s eau 3 poireau   400 gramme lardon   100 gramme fromage raper   25 cl creme frais 3 oeuf',
 '1 oeuf plus etre gourmand faire plat',
 '  100 gramme lardon lamelle   200 gramme farine   1 sachet levure chimique   7 cl huile olive   10 cl lait   1 noisette beurre   moule 1 tomate couper de   100 gramme de jambon 3 oeuf',
 '  1 beau bouquet persil 5 oignon jaune   10 cl vin blanc   40 gramme beurre 18 coquille saint-jacques frais surgelee',
 '  500 gramme coulemelle lepiote eleve fraichement recolter',
 '

In [9]:
eval(data["Etapes"][0])

[{'Étape 1': "Enlever l'écorce et les pépins du potimarron (il n'est pas obligatoire de le peler, mais dans ce cas, le choisir bio et bien le laver) puis couper la chair en gros morceaux."},
 {'Étape 2': 'Eplucher les pommes de terre puis les couper en gros morceaux.'},
 {'Étape 3': "Faire revenir les oignons et l'ail hachés dans un peu de beurre à feu doux."},
 {'Étape 4': 'Ajouter les pommes de terre et le potimarron, faire revenir 5 min.'},
 {'Étape 5': "Couvrir le tout d'eau, ajouter le bouillon de poule (émietté), le curry, la muscade, la feuille de laurier et laisser mijoter environ 30 min à feu doux."},
 {'Étape 6': "Lorsque les légumes sont cuits (vérifier à l'aide d'un couteau), mixer votre préparation."},
 {'Étape 7': "Vérifier l'assaisonnement, saler, poivrer selon votre goût puis ajouter la crème fraîche et laisser mijoter encore 2 ou 3 min. Bon appétit !"}]